In [16]:
import wandb
wandb.login()

True

## Using Adam Optimizer

In [19]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the directory where the data is stored
# Replace with the path to your dataset
data_dir = '/Users/mrinoybanerjee/Desktop/Duke/Classes/Spring 2024/AIPI 540 - Deep Learning/Wound-Detection/Data/Wound_dataset copy'

# Define transforms for the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)

#with wandb.init(project='wound-detection'):
# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% of dataset
val_size = len(dataset) - train_size  # 20% of dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Load a pretrained resnet50 model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a layer tailored to the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

# Move the model to the GPU if available
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # log results
        #wandb.log({'epoch': epoch, 'training_loss': loss})

    # Validation pass
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).type(torch.FloatTensor)
            accuracy += correct.mean()
            #wandb.log({'epoch': epoch, 'validation_loss': val_loss, 'accuracy': accuracy})

    print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss/len(trainloader):.3f}.. "
        f"Validation loss: {val_loss/len(valloader):.3f}.. "
        f"Validation accuracy: {accuracy/len(valloader):.3f}")

# Save the model if you need to use it later
torch.save(model.state_dict(), '/Users/mrinoybanerjee/Desktop/Duke/Classes/Spring 2024/AIPI 540 - Deep Learning/Wound-Detection/Models/wound_model.pth')
#wandb.save('wound_model_1.pth')


/Users/mrinoybanerjee/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mrinoybanerjee/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10.. Train loss: 1.705.. Validation loss: 1.149.. Validation accuracy: 0.605
Epoch 2/10.. Train loss: 1.091.. Validation loss: 0.941.. Validation accuracy: 0.647
Epoch 3/10.. Train loss: 0.981.. Validation loss: 0.863.. Validation accuracy: 0.715
Epoch 4/10.. Train loss: 0.926.. Validation loss: 0.850.. Validation accuracy: 0.728
Epoch 5/10.. Train loss: 0.809.. Validation loss: 0.784.. Validation accuracy: 0.717
Epoch 6/10.. Train loss: 0.799.. Validation loss: 0.961.. Validation accuracy: 0.637
Epoch 7/10.. Train loss: 0.765.. Validation loss: 0.867.. Validation accuracy: 0.681
Epoch 8/10.. Train loss: 0.772.. Validation loss: 1.026.. Validation accuracy: 0.666
Epoch 9/10.. Train loss: 0.770.. Validation loss: 0.727.. Validation accuracy: 0.741
Epoch 10/10.. Train loss: 0.685.. Validation loss: 0.724.. Validation accuracy: 0.746


## Using One cycle policy

In [15]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#with wandb.init(project='wound-detection'):

# Define the directory where the data is stored
# Replace with the path to your dataset
data_dir = '/Users/mrinoybanerjee/Desktop/Duke/Classes/Spring 2024/AIPI 540 - Deep Learning/Wound-Detection/Data/Wound_dataset copy'

# Define transforms for the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% of dataset
val_size = len(dataset) - train_size  # 20% of dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Load a pretrained resnet50 model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a layer tailored to the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

# Move the model to the GPU if available
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# use one cycle learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(trainloader), epochs=10)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # log results
        #wandb.log({'epoch': epoch, 'training_loss': loss})

    # Validation pass
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).type(torch.FloatTensor)
            accuracy += correct.mean()
            #wandb.log({'epoch': epoch, 'validation_loss': val_loss, 'accuracy': accuracy})

    print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss/len(trainloader):.3f}.. "
        f"Validation loss: {val_loss/len(valloader):.3f}.. "
        f"Validation accuracy: {accuracy/len(valloader):.3f}")
    
# Save the model if you need to use it later
torch.save(model.state_dict(), './Models/wound_model.pth')
#wandb.save('wound_model_1.pth')


/Users/mrinoybanerjee/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mrinoybanerjee/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10.. Train loss: 1.777.. Validation loss: 1.556.. Validation accuracy: 0.529


KeyboardInterrupt: 